In [53]:
import numpy as np
import open3d as o3d
import pandas as pd

# %matplotlib inline
# %config InlineBackend.figure_format = 'svg'

import matplotlib
import matplotlib.pyplot as plt
import warnings
import os
import re
import joblib
import os.path as osp

from tqdm import  tqdm
from radar_preprocess.radar_dataclass import *
from natsort import natsorted
from radar_preprocess.radar_data_preprocess import RadarData
from typing import List, Dict


plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'

warnings.filterwarnings('ignore')

# Temploral align

In [54]:
LIDAR_FILE_ROOT_PATH = '/Users/austin/Downloads/VRFusion/lidar/chengdu-2025-02-26/data'
RADAR_FILE_ROOT_PATH = '/Users/austin/Downloads/VRFusion/radar/chengdu-2025-02-26/json'

In [55]:
import os
import re
from natsort import natsorted
import numpy as np


def extract_timestamp(filename):
    match = re.search(r'(\d+\.\d+)', filename)
    if match:
        return float(match.group(1))
    return None

def extract_timestamp_json(filename):
    match = re.search(r'(\d+\_\d+)', filename)
    if match:
        return float(match.group(1)) / 1000
    return None

lidar_file_list = natsorted(os.listdir(LIDAR_FILE_ROOT_PATH))
radar_file_list = natsorted(os.listdir(RADAR_FILE_ROOT_PATH))

lidar_file_list = [f for f in lidar_file_list if f.endswith('.pcd')]
radar_file_list = [f for f in radar_file_list if f.endswith('.json')]

pcd_files  = sorted(lidar_file_list, key=lambda x: extract_timestamp(x))
json_files = sorted(radar_file_list, key=lambda x: extract_timestamp_json(x))

aligned_data_dict = dict()

lidar_align_list = list()
radar_align_list = list()


last_closest_json_file = None

for pcd_file in pcd_files:
    pcd_timestamp = extract_timestamp(pcd_file)
    closest_json_file = None
    min_time_diff = float('inf')

    for json_file in json_files:
        json_timestamp = extract_timestamp_json(json_file)
        time_diff = abs(pcd_timestamp - json_timestamp)

        if time_diff < min_time_diff:
            min_time_diff = time_diff
            closest_json_file = json_file

    if closest_json_file and closest_json_file != last_closest_json_file:
        aligned_data_dict[pcd_file] = closest_json_file
        lidar_align_list.append(osp.join(LIDAR_FILE_ROOT_PATH, pcd_file))
        radar_align_list.append(osp.join(RADAR_FILE_ROOT_PATH, closest_json_file))

        last_closest_json_file = closest_json_file

# for pcd_file in aligned_data_dict:
#     print(f'PCD File: {pcd_file} -> JSON File: {aligned_data_dict[pcd_file]}')

aligned_data_path = '/Users/austin/Downloads/VRFusion/result/aligned_data'

joblib.dump([lidar_align_list[:-1],radar_align_list[:-1] ], 
            osp.join(aligned_data_path, 
                     LIDAR_FILE_ROOT_PATH.split('/')[6] + ':lidar_radar_temploral_aligned_data.list'))

['/Users/austin/Downloads/VRFusion/result/aligned_data/chengdu-2025-02-26:lidar_radar_temploral_aligned_data.list']

In [56]:
temporal_aligned_data = joblib.load('/Users/austin/Downloads/VRFusion/result/aligned_data/chengdu-2025-02-26:lidar_radar_temploral_aligned_data.list')
print('\'{}\''.format(temporal_aligned_data[0][200]))
print('\'{}\''.format(temporal_aligned_data[1][200]))

'/Users/austin/Downloads/VRFusion/lidar/chengdu-2025-02-26/data/1740546284.767191000.pcd'
'/Users/austin/Downloads/VRFusion/radar/chengdu-2025-02-26/json/1740546284_745.json'


In [57]:
for i in range(len(temporal_aligned_data[0])):
    print(temporal_aligned_data[0][i], temporal_aligned_data[1][i])

/Users/austin/Downloads/VRFusion/lidar/chengdu-2025-02-26/data/1740546244.765178000.pcd /Users/austin/Downloads/VRFusion/radar/chengdu-2025-02-26/json/1740546244_753.json
/Users/austin/Downloads/VRFusion/lidar/chengdu-2025-02-26/data/1740546244.965956000.pcd /Users/austin/Downloads/VRFusion/radar/chengdu-2025-02-26/json/1740546244_953.json
/Users/austin/Downloads/VRFusion/lidar/chengdu-2025-02-26/data/1740546245.166456000.pcd /Users/austin/Downloads/VRFusion/radar/chengdu-2025-02-26/json/1740546245_153.json
/Users/austin/Downloads/VRFusion/lidar/chengdu-2025-02-26/data/1740546245.365258000.pcd /Users/austin/Downloads/VRFusion/radar/chengdu-2025-02-26/json/1740546245_353.json
/Users/austin/Downloads/VRFusion/lidar/chengdu-2025-02-26/data/1740546245.565276000.pcd /Users/austin/Downloads/VRFusion/radar/chengdu-2025-02-26/json/1740546245_553.json
/Users/austin/Downloads/VRFusion/lidar/chengdu-2025-02-26/data/1740546245.765449000.pcd /Users/austin/Downloads/VRFusion/radar/chengdu-2025-02-26

# Spatial align

In [58]:
idx = 3

LIDAR_FILE_PATH = temporal_aligned_data[0][idx]
LIDAR_ANNO_PATH = '/Users/austin/Downloads/VRFusion/result/annotation/chengdu-2025-02-26:lidar_annotation.feather'
RADAR_FILE_PATH = temporal_aligned_data[1][idx]

In [59]:
timestamp = LIDAR_FILE_PATH.split('/')[-1].split('.')[-3] + LIDAR_FILE_PATH.split('/')[-1].split('.')[-2][:-4]

anno_df = pd.read_feather(LIDAR_ANNO_PATH)
anno_df = anno_df[anno_df['Timestamp'] == timestamp]
anno_df

,Timestamp,ID,Lon,Lat,x_size,y_size,z_size,Ori,x,y,z
92,174054624536525,3.0,104.074434,30.427527,4.459859,1.897667,1.616043,66.787148,72.224533,-27.588593,1.523438
93,174054624536525,20.0,104.073765,30.427113,4.420833,1.810766,1.580536,-84.289032,138.795624,14.687187,1.273438
94,174054624536525,4448.0,104.073425,30.427407,4.614931,1.860054,1.625143,86.214035,169.715546,-19.588127,1.461914
95,174054624536525,5579.0,104.075584,30.427610,0.754471,0.585578,1.777478,-101.308594,-38.491089,-30.783749,1.668945
96,174054624536525,5787.0,104.073526,30.427337,4.605926,1.829429,1.607779,91.467377,160.424835,-11.310703,1.425781
97,174054624536525,5815.0,104.075585,30.427601,0.639686,0.567561,1.716912,-88.425255,-38.593750,-29.760624,1.588867
98,174054624536525,5834.0,104.076315,30.427405,4.429476,1.862781,1.666528,-80.803421,-107.436089,-4.312580,1.637695
99,174054624536525,5841.0,104.076019,30.427221,1.902305,0.813193,1.528545,-90.758789,-77.965546,14.499138,1.370117
100,174054624536525,5894.0,104.075774,30.427582,4.268113,1.762784,1.570151,100.895905,-56.597183,-26.718515,1.514648
101,174054624536525,5903.0,104.074512,30.427522,4.587969,1.863690,1.648319,60.750923,64.818451,-26.673748,1.566406


In [60]:
data = RadarData(root_path=RADAR_FILE_ROOT_PATH, duration_frames=1)
dets_list: List[detections] = data.read_single_frame(data_file_path=RADAR_FILE_PATH).dets.det_list

Loading raw RADAR frames...: 100%|██████████| 1/1 [00:00<00:00, 10.00it/s]


In [61]:
dets_array = np.asarray([np.array([det.add_pos_x, -det.add_pos_y, 0]) for det in dets_list])

theta = 2 * np.pi / 180
rotation_array = np.array([[ np.cos(theta), np.sin(theta), 0],
                           [-np.sin(theta), np.cos(theta), 0],
                           [             0,             0, 0]])

radar_point_clouds = dets_array.reshape((-1, 3)) @ rotation_array + np.asarray([0, 21, 0])
radar_pcd = o3d.geometry.PointCloud()
radar_pcd.points = o3d.utility.Vector3dVector(radar_point_clouds)
radar_pcd.paint_uniform_color([1, 0, 0])

PointCloud with 216 points.

In [62]:
def create_bounding_box(center, size, rotation_angle, do_init=False):

    if do_init:
        return o3d.geometry.TriangleMesh.create_box()

    box = o3d.geometry.TriangleMesh.create_box(width=size[0], height=size[1], depth=size[2])
    rotation_matrix = o3d.geometry.get_rotation_matrix_from_xyz([0, 0, rotation_angle])

    box.rotate(rotation_matrix, center=(0, 0, 0))
    box.translate(center)
    
    return box

In [63]:
def create_bounding_box_lines(center, size, rotation_angle):
    # 计算立方体的8个顶点
    half_size = np.array(size) / 2
    corners = np.array([
        [-half_size[0], -half_size[1], -half_size[2]],
        [ half_size[0], -half_size[1], -half_size[2]],
        [ half_size[0],  half_size[1], -half_size[2]],
        [-half_size[0],  half_size[1], -half_size[2]],
        [-half_size[0], -half_size[1],  half_size[2]],
        [ half_size[0], -half_size[1],  half_size[2]],
        [ half_size[0],  half_size[1],  half_size[2]],
        [-half_size[0],  half_size[1],  half_size[2]]
    ])

    # 创建旋转矩阵
    rotation_matrix = o3d.geometry.get_rotation_matrix_from_xyz([0, 0, (rotation_angle - 90) * np.pi / 180])
    corners = np.dot(corners, rotation_matrix.T) + center

    # 定义边的连接（8个顶点，12条边）
    lines = [
        [0, 1], [1, 2], [2, 3], [3, 0],  # 底面
        [4, 5], [5, 6], [6, 7], [7, 4],  # 顶面
        [0, 4], [1, 5], [2, 6], [3, 7]   # 连接底面和顶面
    ]

    # 创建 LineSet 并添加点和线
    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(corners)
    line_set.lines = o3d.utility.Vector2iVector(lines)

    return line_set

In [64]:
# pcd = o3d.io.read_point_cloud(LIDAR_FILE_PATH)

# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name='Point Cloud Visualization', width=800, height=600, left=50, top=50)

# render_option = vis.get_render_option()
# render_option.point_size = 2

# vis.add_geometry(pcd)
# vis.add_geometry(radar_pcd)

# for idx in range(anno_df.shape[0]):
#     anno_series = anno_df.iloc[idx]
#     bbox = create_bounding_box_lines(center=[anno_series['x'], anno_series['y'], anno_series['z']],
#                                size=[anno_series['x_size'], anno_series['y_size'], anno_series['z_size']],
#                                rotation_angle=anno_series['Ori'])
#     vis.add_geometry(bbox)

# vis.run()
# vis.destroy_window()

In [65]:
# plt.figure(figsize=(16, 16))
# plt.scatter(dets_array[:, 0], dets_array[:, 1], s=0.2)

In [66]:
# def load_pcd(file_path):
#     # 使用 open3d 读取二进制格式的 PCD 文件
#     pcd = o3d.io.read_point_cloud(file_path)
#     points = np.asarray(pcd.points)
#     return points

In [67]:
# lidar_point_clounds = load_pcd(LIDAR_FILE_PATH)

In [68]:
# plt.figure(figsize=(16, 16))
# plt.scatter(lidar_point_clounds[:, 0], lidar_point_clounds[:, 1], s=0.1)

In [69]:
# plt.figure(figsize=(16, 16))
# plt.scatter(lidar_point_clounds[:, 0], lidar_point_clounds[:, 1], s=0.1)
# plt.scatter(dets_array[:, 0], -dets_array[:, 1]+24, s=0.2)